In [1]:
! pip -q install pytorch_lightning transformers wandb sentence-transformers 

     |████████████████████████████████| 705 kB 9.4 MB/s 
     |████████████████████████████████| 4.7 MB 56.4 MB/s 
     |████████████████████████████████| 1.8 MB 52.3 MB/s 
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 5.9 MB 47.7 MB/s 
     |████████████████████████████████| 419 kB 55.4 MB/s 
     |████████████████████████████████| 6.6 MB 50.0 MB/s 
     |████████████████████████████████| 120 kB 38.7 MB/s 
     |████████████████████████████████| 122 kB 51.3 MB/s 
     |████████████████████████████████| 181 kB 56.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 157 kB 70.4 MB/s 
     |████████████████████████████████| 157 kB 78.0 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 76.6 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 156 kB 78.5 MB/s 
     |█████████████

In [2]:
import warnings 
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np
import glob
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import re
import math
import gc
from tqdm.notebook import tqdm_notebook
import wandb

import torch
import torch.nn as nn 
import torch.nn.functional as F
import transformers

import pytorch_lightning as pl
import pytorch_lightning

from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import CrossEncoder

device =  torch.device( 'cpu')
device

device(type='cpu')

In [3]:
# load the models 
! wget -q "https://www.dropbox.com/s/58ns2jt8aj8yoxm/models.zip?dl=0"
! unzip -q models.zip?dl=0

# load python dataset
! wget -q "https://www.dropbox.com/s/eijnvwna66sq0rq/python.zip?dl=0"
! unzip -q python.zip?dl=0

# load javascript dataset
! wget -q "https://www.dropbox.com/s/rmzuylbwpj2l44y/javascript.zip?dl=0"
! unzip -q javascript.zip?dl=0

# Load Dataset

In [20]:
# please select between python or java script 

language = int( input("0: for python \n1: for javascript\n") )


if language == 0:
  dir = "./python/"
  load_model = "/content/models/python_model_2.ckpt"
else:
  dir = "./javascript/"
  load_model = "/content/models/js_model.ckpt"


0: for python 
1: for javascript
0


In [5]:

def jsonl_list_to_dataframe(file_list, dir , columns=['code', 'docstring']):
    
    """
    input  : Directory 
    output : Dataframe that has code and docstring 
    """

    file_list =   glob.glob(f"{dir}/**.gz", recursive= True)

    return pd.concat([pd.read_json( f, orient='records', compression='gzip', lines=True)[columns] for f in file_list], sort=False)

def get_dfs(path):
    """
    input  : Directory 
    output : returns train, Validation and test  dataframe
    """
    
    dfs = []
    for split in ["test", "valid", "test"]:
        
        files = sorted( os.listdir( path+split  ) )
        df = jsonl_list_to_dataframe(files, path+split ) #.rename(columns = {'code': 'mthd', 'docstring': 'cmt'})
        dfs.append( df )
        
    return dfs

train, valid, test = get_dfs(dir)

valid.shape, test.shape

((23107, 2), (22176, 2))

In [6]:
valid.head()

,code,docstring
0,"def learn(env,\n network,\n ...",Train a deepq model.\n\n Parameters\n --...
1,"def save_act(self, path=None):\n """"""Sav...",Save model to a pickle located at `path`
2,"def nature_cnn(unscaled_images, **conv_kwargs)...",CNN from Nature paper.
3,"def mlp(num_layers=2, num_hidden=64, activatio...",Stack of fully-connected layers to be used in ...
4,"def lstm(nlstm=128, layer_norm=False):\n """"...",Builds LSTM (Long-Short Term Memory) network t...


# Data Cleaning  

In [7]:
def del_docstring_from_code(value):
    """
    input : code and docstring
    output clean code snippet  
    """
    if value[1] in value[0]:
        value[0] = value[0].replace(value[1] , "")


    return value[0]



test.code  = [ del_docstring_from_code(value) for value in test.values]
valid.code = [ del_docstring_from_code(value) for value in valid.values] 


valid.shape, test.shape

((23107, 2), (22176, 2))

In [8]:
# remove html. tag from the code 

def remove_html( cmt ):
    """
    input  : Docstring
    output : Clean comment  
    """
    return re.sub(r"<.?span[^>]*>|<.?code[^>]*>|<.?p[^>]*>|<.?hr[^>]*>|<.?h[1-3][^>]*>|<.?a[^>]*>|<.?b[^>]*>|<.?blockquote[^>]*>|<.?del[^>]*>|<.?dd[^>]*>|<.?dl[^>]*>|<.?dt[^>]*>|<.?em[^>]*>|<.?i[^>]*>|<.?img[^>]*>|<.?kbd[^>]*>|<.?li[^>]*>|<.?ol[^>]*>|<.?pre[^>]*>|<.?s[^>]*>|<.?sup[^>]*>|<.?sub[^>]*>|<.?strong[^>]*>|<.?strike[^>]*>|<.?ul[^>]*>|<.?br[^>]*>", "", cmt )


test.docstring = test.docstring.apply(lambda x : remove_html(x))
valid.docstring = valid.docstring.apply(lambda x : remove_html(x))


valid.shape, test.shape

((23107, 2), (22176, 2))

In [9]:
# get rid of the rowes that has < code > tag in there docstring 

def has_code(cmt) :
    
    '''
    Determinine if the given comment contains the HTML <code> tag
    '''

    if '<code>' in cmt: return True
    else: return False



test  = test[ ~ test.docstring.apply(lambda x : has_code(x) ) ]
valid = valid[ ~ valid.docstring.apply(lambda x : has_code(x) ) ]


valid.shape, test.shape

((23107, 2), (22176, 2))

In [10]:
# delet the rows where the length of the docstring is longer than the length of the code! This is becase some time code comment tend to have extra 
# info which is not available in real time 




test = test[ ~ test.apply(lambda row : len(row.code) < len(row.docstring) , axis= 1 ) ]

valid = valid[ ~ valid.apply(lambda row : len(row.code) < len(row.docstring) , axis= 1 ) ] 

valid.shape, test.shape

((18428, 2), (18373, 2))

In [11]:
# Remove the rows that has missing docstring  


test  = test[ ~ ( test.docstring == '') ]
valid = valid[ ~ ( valid.docstring == '') ]


valid.shape, test.shape

((18428, 2), (18373, 2))

In [12]:
# remove duplicate values 


test = test[ ~ test.docstring.duplicated() ]
valid = valid[ ~ valid.docstring.duplicated() ]

valid.shape, test.shape

((17736, 2), (17756, 2))

In [13]:

test = test.reset_index().drop(labels = 'index', axis = 1)
valid = valid.reset_index().drop(labels = 'index', axis = 1)

# Tokenization

In [14]:
# initialise our tokenizer 

tokenizer = transformers.AutoTokenizer.from_pretrained("microsoft/codebert-base")

In [15]:
# lets only consider the average length of the code and comment 

code_max_length, cmt_max_length =  256  ,  64      # "int(np.average(code_lengths))", int(np.average(cmt_lengths)) 

code_max_length, cmt_max_length 



BATCH_SIZE = 32

## Develop model

In [16]:
class Beam(object):
    def __init__(self, size,sos,eos):
        self.size = size
        self.tt = torch
        # The score for each translation on the beam.
        self.scores = self.tt.FloatTensor(size).zero_()
        # The backpointers at each time-step.
        self.prevKs = []
        # The outputs at each time-step.
        self.nextYs = [self.tt.LongTensor(size)
                       .fill_(0)]
        self.nextYs[0][0] = sos
        # Has EOS topped the beam yet.
        self._eos = eos
        self.eosTop = False
        # Time and k pair for finished.
        self.finished = []

    def getCurrentState(self):
        "Get the outputs for the current timestep."
        batch = self.tt.LongTensor(self.nextYs[-1]).view(-1, 1)
        return batch

    def getCurrentOrigin(self):
        "Get the backpointers for the current timestep."
        return self.prevKs[-1]

    def advance(self, wordLk):
        """
        Given prob over words for every last beam `wordLk` and attention
        `attnOut`: Compute and update the beam search.
        Parameters:
        * `wordLk`- probs of advancing from the last step (K x words)
        * `attnOut`- attention at the last step
        Returns: True if beam search is complete.
        """
        numWords = wordLk.size(1)

        # Sum the previous scores.
        if len(self.prevKs) > 0:
            beamLk = wordLk + self.scores.unsqueeze(1).expand_as(wordLk)

            # Don't let EOS have children.
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] == self._eos:
                    beamLk[i] = -1e20
        else:
            beamLk = wordLk[0]
        flatBeamLk = beamLk.view(-1)
        bestScores, bestScoresId = flatBeamLk.topk(self.size, 0, True, True)

        self.scores = bestScores

        # bestScoresId is flattened beam x word array, so calculate which
        # word and beam each score came from
        prevK = bestScoresId // numWords
        self.prevKs.append(prevK)
        self.nextYs.append((bestScoresId - prevK * numWords))


        for i in range(self.nextYs[-1].size(0)):
            if self.nextYs[-1][i] == self._eos:
                s = self.scores[i]
                self.finished.append((s, len(self.nextYs) - 1, i))

        # End condition is when top-of-beam is EOS and no global score.
        if self.nextYs[-1][0] == self._eos:
            self.eosTop = True

    def done(self):
        return self.eosTop and len(self.finished) >=self.size

    def getFinal(self):
        if len(self.finished) == 0:
            self.finished.append((self.scores[0], len(self.nextYs) - 1, 0))
        self.finished.sort(key=lambda a: -a[0])
        if len(self.finished) != self.size:
            unfinished=[]
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] != self._eos:
                    s = self.scores[i]
                    unfinished.append((s, len(self.nextYs) - 1, i)) 
            unfinished.sort(key=lambda a: -a[0])
            self.finished+=unfinished[:self.size-len(self.finished)]
        return self.finished[:self.size]

    def getHyp(self, beam_res):
        """
        Walk back to construct the full hypothesis.
        """
        hyps=[]
        for _,timestep, k in beam_res:
            hyp = []
            for j in range(len(self.prevKs[:timestep]) - 1, -1, -1):
                hyp.append(self.nextYs[j+1][k])
                k = self.prevKs[j][k]
            hyps.append(hyp[::-1])
        return hyps
    
    def buildTargetTokens(self, preds):
        sentence=[]
        for pred in preds:
            tokens = []
            for tok in pred:
                if tok==self._eos:
                    break
                tokens.append(tok)
            sentence.append(tokens)
        return sentence

In [17]:
class Code_Comment_Generator(pl.LightningModule):

    def __init__(self, encoder, decoder, d_model, src_vocab_size, tgt_vocab_Size, pad_idx, dropout, src_max_length, tgt_max_length, config, lr, epoch, tokenizer ):

        super(Code_Comment_Generator, self).__init__() 

        
        self.d_model = d_model
        self.pad_idx = pad_idx
        self.lr = lr
        self.epoch = epoch
        self.tgt_max_length = tgt_max_length #
        self.beam_size = 10
        self.sos_id = 0
        self.eos_id =2
        self.tokenizer = tokenizer

        self.test_cases = []        
        self.eval_counter = 0

        # pretrained config 
        self.config = config

        # Roberta Encoder  layer 
        self.encoder = encoder

        # Decoder Transformers 
        self.decoder = decoder


        # Register buffer 
        self.register_buffer("bias", torch.tril(torch.ones(2048, 2048))) # This is typically used to register a buffer that should not to be considered a model parameter.

        # dense layers 
        self.dense_layer = nn.Linear(in_features= self.d_model, 
                                out_features= self.d_model)


        # generator
        self.generator = nn.Linear(in_features= self.d_model, 
                                out_features= tgt_vocab_Size,  bias=False)

        self.dropout = nn.Dropout(p=dropout)

        # final activation 
        self.log_softmax = nn.LogSoftmax(dim= -1)
        
        # sharing weights between the encoder and the decoder 
        self.share_weights() 

        

    def share_weights(self):
        """ 
            This is to ensure we are sharing weights between the encoder and fianl dense layer 
        """

        if self.config.torchscript:
            self.generator.weight = nn.Parameter( data =  self.encoder.embeddings.word_embeddings.weight.clone()) 
        
        else: 
            self.generator.weight = self.encoder.embeddings.word_embeddings.weight

    
    def forward(self, src = None, src_key_padding_mask= None, tgt= None, tgt_key_padding_mask= None):
        """ 
        input  : source and taget tokens and attention mask 
        output : logits

        """


        # encoder output, this gives us the context vector which is present in the last decoder layer 

        encoder_output = self.encoder(src, attention_mask = src_key_padding_mask )[0]# considering the output of the last encoder layer  : shape -> torch.Size([32, 212, 768]) -> [N,S,E]
        encoder_output = encoder_output.permute([1,0,2]).contiguous() # rearanging tensor to shape -> torch.Size([212,32,768]) -> [S,N,E] 

        if tgt is not None: 

          tgt_mask = -1e4 *(1-self.bias[:tgt.shape[1],:tgt.shape[1]]).to(device= device) # shape -> trg_mask > torch.Size([42, 42])
      
          # encode the target embeddings 
          tgt_embedding = self.encoder.embeddings(tgt)                    # shape: torch.Size([32, 42, 768]) -> [N, T, E]
          tgt_embedding = tgt_embedding.permute([1,0,2]).contiguous()     # shape: torch.Size([42, 32, 768]) -> [T, N, E]

          # memeory key padding mask 
          memory_key_padding_mask = (1-src_key_padding_mask).bool()      # shape : torch.Size([32, 212])     -> [N, E]
        
        
          # output of the decoder 
          output = self.decoder(tgt = tgt_embedding, 
                                  memory = encoder_output, 
                                  tgt_mask = tgt_mask, 
                                  memory_key_padding_mask = memory_key_padding_mask )               # torch.Size([42, 32, 768]) -> T, N, E




          output =    torch.tanh( self.dropout(   self.dense_layer(output)  ) ).permute([1,0,2]).contiguous()  # activation + FC_1         # torch.Size([42, 32, 768])  -> T, N, E

          output =  self.generator(output)                    # output shape :- torch.Size([42, 32, 50265]) -> T, N, TGT_Vocb_size


          # calculate loss 
          active_loss = tgt_key_padding_mask[: , 1: ].ne(0).view(-1) == 1      # shape -> torch.Size([1312]) N*(T-1)

          shift_logits = output[:, :-1, :].contiguous()                        # shape -> torch.Size([32, 41, 50265]) N, (T-1), TGT_vocab_size

          shift_labels = tgt[:, 1:].contiguous()                            # shape -> torch.Size([32, 41])        N, (T-1)


          # flatten the tokens and find the loss 
          y_hat = shift_logits.view(-1, shift_logits.shape[-1])[active_loss]  # predicted value flatened
          y     = shift_labels.view(-1)[active_loss]                          # actual value flatten

          # find the cross entropy loss 
          loss = F.cross_entropy(y_hat ,  y , ignore_index = self.pad_idx )

          # memory management 
          del output
          torch.cuda.empty_cache()
          gc.collect()
          pl.utilities.memory.garbage_collection_cuda()
          pl.utilities.memory.get_gpu_memory_map()
          

          return loss #, loss*active_loss.sum() # train loss, evaluation loss 
        
        else: 
          # prediction 
          preds = []

          

          # zero = torch.cuda.LongTensor(1).fill_(0)

          zero = torch.LongTensor(1).fill_(0)

          for i in range(src.shape[0]):

            context = encoder_output[:,i:i+1]

            context_mask = src_key_padding_mask[i:i+1,:]

            beam = Beam(self.beam_size, self.sos_id, self.eos_id) ####

            input_ids=beam.getCurrentState()

            context=context.repeat(1, self.beam_size,1)

            context_mask=context_mask.repeat(self.beam_size,1)     

            for _ in range(self.tgt_max_length):   #######
              
              if beam.done():
                break

              tgt_mask=-1e4 *(1-self.bias[:input_ids.shape[1],:input_ids.shape[1]]) 

              tgt_embedding = self.encoder.embeddings(input_ids).permute([1,0,2]).contiguous() 

              # memeory key padding mask 
              memory_key_padding_mask = (1-context_mask).bool()      # shape : torch.Size([32, 212])     -> [N, E]
            
            
              # output of the decoder 
              output = self.decoder(tgt = tgt_embedding, 
                                      memory = context, 
                                      tgt_mask = tgt_mask, 
                                      memory_key_padding_mask = memory_key_padding_mask )               # torch.Size([42, 32, 768]) -> T, N, E




              output =  torch.tanh( self.dropout(   self.dense_layer(output)  ) ).permute([1,0,2]).contiguous()[:,-1,:]  # activation + FC_1         # torch.Size([42, 32, 768])  -> T, N, E

              output =  self.log_softmax( self.generator(output) ).data               # output shape :- torch.Size([42, 32, 50265]) -> T, N, TGT_Vocb_size

              beam.advance(output)

              input_ids.data.copy_(input_ids.data.index_select(0, beam.getCurrentOrigin()))
              input_ids=torch.cat((input_ids,beam.getCurrentState()),-1)

            hyp= beam.getHyp(beam.getFinal())
            pred=beam.buildTargetTokens(hyp)[:self.beam_size]
            pred=[torch.cat([x.view(-1) for x in p]+[zero]*(self.tgt_max_length -len(p))).view(1,-1) for p in pred]
            preds.append(torch.cat(pred,0).unsqueeze(0))
          
          preds=torch.cat(preds,0)   

          del output
          del context

          

          return preds


    def training_step(self, batch, batch_indx): 


        # train model
        self.train()

        # unrap and load all the trensors to device
        src, src_key_padding_mask, tgt, tgt_key_padding_mask = tuple( i.to(device) for i in batch)

        #returns loss
        Loss  = self( src, src_key_padding_mask, tgt, tgt_key_padding_mask )

        # log the loss 
        self.log(name= "tran_loss", value= Loss, on_step= True, on_epoch=True, prog_bar= True, logger= True)

        # memory management
        gc.collect()
        pl.utilities.memory.garbage_collection_cuda()
        pl.utilities.memory.get_gpu_memory_map()

        return Loss

    def validation_step(self, batch, batch_indx): 


        # evaluate model
        self.eval()

        if self.eval_counter == 400 : # self.train_dataloader().batch_size

          code = " def search_features(self, search): if isinstance(search, string_types): search = [search] search = [s.replace('*', '.*') for s in search] cols = list(self.data.columns) results = [] for s in search: results.extend([f for f in cols if re.match(s + '$', f)]) return list(set(results)) " 
          doc = "returns all features that match any of the elements in the input list."

          print(f"generated cmt : {self.generate_comment(code) }")
          print(f"original cmt : {doc}")  

          self.test_cases.append(self.generate_comment(code))

          self.eval_counter = 0


        self.eval_counter += 1

        # unrap and load all the trensors to device
        src, src_key_padding_mask, tgt, tgt_key_padding_mask = tuple( i.to(device) for i in batch)

        #returns loss
        Loss  = self( src, src_key_padding_mask, tgt, tgt_key_padding_mask )


        # log the loss 
        self.log(name= "validation_loss", value= Loss, on_step= True, on_epoch=True, prog_bar= True, logger= True)

        # memory management
        gc.collect()
        pl.utilities.memory.garbage_collection_cuda()
        pl.utilities.memory.get_gpu_memory_map()

        return Loss


    def configure_optimizers(self):


        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']

        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': 0.0 },
            {'params': [p for n, p in self.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr = self.lr , eps= 1e-8 )
        
        nn.utils.clip_grad_norm_(self.parameters(), 1.0)

        torch.autograd.set_detect_anomaly(True)

        lr_scheduler = {
            'scheduler' : transformers.get_linear_schedule_with_warmup(optimizer= optimizer, num_warmup_steps = 5000, num_training_steps= self.epoch * len(Test)  ), 
            'interval'  : "step", 
            'frequency' : 1, 
            'monitor'   : "validation_loss", 
        }



        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}


    def generate_comment(self, code):
      """
      input : single code string 
      output : Generated comment 
      """

      tokenized = self.tokenizer.encode_plus(code, max_length= 256, padding='max_length', return_attention_mask= True, truncation= True )

      src = torch.LongTensor(tokenized['input_ids'] ).unsqueeze(1).reshape(1,-1).to(device)
      src_key_padding_mask = torch.LongTensor(tokenized['attention_mask'] ).unsqueeze(1).reshape(1,-1).to(device)

      gen_cmt = []

      preds = self(src, src_key_padding_mask)

      for pred in preds:
        t  = pred[0].cpu().numpy()
        t=list(t)
        if 0 in t:
          t=t[:t.index(0)]
        text = self.tokenizer.decode(t,clean_up_tokenization_spaces=False)
        
        gen_cmt.append(text)

      return gen_cmt
 

## Load the model pre-trained model

In [18]:
#pretrained configuration
config = transformers.RobertaConfig.from_pretrained("microsoft/codebert-base")

d_model = config.hidden_size
src_vocab_size = tokenizer.vocab_size
tgt_vocab_Size = tokenizer.vocab_size
pad_idx = tokenizer.pad_token_id
dropout = 0.5
src_max_length = code_max_length
tgt_max_length = cmt_max_length
epoch = 1000
lr = 5e-5

# Roberta Encoder  layer 
encoder = transformers.RobertaModel.from_pretrained(pretrained_model_name_or_path= "microsoft/codebert-base", config =  config ) 


# Decoder Transformers 
decoder_layer = torch.nn.TransformerDecoderLayer(d_model= config.hidden_size, nhead= config.num_attention_heads)
decoder = torch.nn.TransformerDecoder(decoder_layer= decoder_layer, num_layers= 6)

# lightning model 
model = Code_Comment_Generator(encoder, decoder, d_model, src_vocab_size, tgt_vocab_Size, pad_idx, dropout, src_max_length, tgt_max_length, config, lr, epoch, tokenizer).to(device)


In [21]:
# Load the pre-trained model

checkpoint = torch.load( load_model , map_location=torch.device('cpu')  )
model.load_state_dict(checkpoint['state_dict']);

In [22]:
# cross encoder to check sentence similarity
check_similarity = CrossEncoder('cross-encoder/stsb-roberta-base')

In [23]:
def code_pre_proceeing(code):
    """
    input  : unprocessed code 
    output : clean code
    """

    # remove any html tag
    code = re.sub(r"<.?span[^>]*>|<.?code[^>]*>|<.?p[^>]*>|<.?hr[^>]*>|<.?h[1-3][^>]*>|<.?a[^>]*>|<.?b[^>]*>|<.?blockquote[^>]*>|<.?del[^>]*>|<.?dd[^>]*>|<.?dl[^>]*>|<.?dt[^>]*>|<.?em[^>]*>|<.?i[^>]*>|<.?img[^>]*>|<.?kbd[^>]*>|<.?li[^>]*>|<.?ol[^>]*>|<.?pre[^>]*>|<.?s[^>]*>|<.?sup[^>]*>|<.?sub[^>]*>|<.?strong[^>]*>|<.?strike[^>]*>|<.?ul[^>]*>|<.?br[^>]*>", "", code)
    
    # remove extra spaces and to lower cases 
    code = ' '.join(code.split()).lower()

    return code

In [24]:
# function to automatically generate comment 

def generated_comment_report(code, actual_cmt = None ):
    """
    input: code snippet that requires comment to be generated 
    output: Generated comments
    """

    # generate comment
    generated_comment = model.generate_comment(code_pre_proceeing(code))[0]

    if actual_cmt is None:
      print( f" Generated Comment  :  \n { generated_comment }" )
    else:
      print( f"  code : \n { code } " )
      print("\n")
      print( f" True comment : \n { actual_cmt  }" )
      print("\n")
      print( f" Generated Comment  :  \n { generated_comment }" )
      print("\n")

      # calculate blue score 
      blue_score = sentence_bleu([actual_cmt.lower()], generated_comment)
      print(f"the blue score for generated comment is {blue_score}")
      print("\n")
      # calculate similarity 
      similarity = np.average( check_similarity.predict([actual_cmt.lower(), generated_comment ] ) )
      print(f"the similarity between sentences  is {similarity}")

In [29]:
# randomly select a record to generate comments and compare them 

idx = np.random.randint(test.shape[0])

code = test.code.values[idx]
cmt = test.docstring.values[idx].split('\n')[0]

generated_comment_report(code, cmt)



  code : 
 def dump(rt, from_date, with_json=True, latest_only=False, **kwargs):
    """
    """
    return dict(id_remote_account=rt.id_remote_account,
                token_type=rt.token_type,
                access_token=rt.access_token,
                secret=rt.secret) 


 True comment : 
 Dump the remote tokens as a list of dictionaries.


 Generated Comment  :  
 dump a token to json.


the blue score for generated comment is 0.13673515160833383


the similarity between sentences  is 0.42471349239349365
